## Installing Dependencies

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install torch

!wget -q https://github.com/huggingface/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
%pip install -qq git+https://github.com/huggingface/diffusers.git
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy xformers bitsandbytes==0.35.0

## Import Packages

In [ ]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms
from __future__ import print_function
import time
from google.colab.patches import cv2_imshow
import requests
from PIL import Image
from transformers import Swin2SRForImageSuperResolution
from transformers import Swin2SRImageProcessor 

## Silhouette

In [ ]:
# Load pretrained model
model = torch.hub.load('pytorch/vision:v0.6.0', 'deeplabv3_resnet101', pretrained=True)
# Segment people only for the purpose of human silhouette extraction
people_class = 15

# Evaluate model
model.eval()
print ("Model has been loaded.")

blur = torch.FloatTensor([[[[1.0, 2.0, 1.0],[2.0, 4.0, 2.0],[1.0, 2.0, 1.0]]]]) / 16.0

# Use GPU if supported, for better performance
if torch.cuda.is_available():
	model.to('cuda')
	blur = blur.to('cuda')
	
# Apply preprocessing (normalization)
preprocess = transforms.Compose([
	transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to create segmentation mask
def makeSegMask(img):
    # Scale input frame
	frame_data = torch.FloatTensor( img ) / 255.0

	input_tensor = preprocess(frame_data.permute(2, 0, 1))
    
    # Create mini-batch to be used by the model
	input_batch = input_tensor.unsqueeze(0)

    # Use GPU if supported, for better performance
	if torch.cuda.is_available():
		input_batch = input_batch.to('cuda')

	with torch.no_grad():
		output = model(input_batch)['out'][0]

	segmentation = output.argmax(0)

	bgOut = output[0:1][:][:]
	a = (1.0 - F.relu(torch.tanh(bgOut * 0.30 - 1.0))).pow(0.5) * 2.0

	people = segmentation.eq( torch.ones_like(segmentation).long().fill_(people_class) ).float()

	people.unsqueeze_(0).unsqueeze_(0)
	
	for i in range(3):
		people = F.conv2d(people, blur, stride=1, padding=1)

	# Activation function to combine masks - F.hardtanh(a * b)
	combined_mask = F.relu(F.hardtanh(a * (people.squeeze().pow(1.5)) ))
	combined_mask = combined_mask.expand(1, 3, -1, -1)

	res = (combined_mask * 255.0).cpu().squeeze().byte().permute(1, 2, 0).numpy()

	return res

In [ ]:
img = cv2.imread('/content/sddefault.png')
# Apply background subtraction to extract foreground (silhouette)
mask = makeSegMask(img)

# Apply thresholding to convert mask to binary map
ret,thresh = cv2.threshold(mask,127,255,cv2.THRESH_BINARY)

# Show extracted silhouette only, by multiplying mask and input frame
final = cv2.bitwise_and(thresh, img)

# Show current frame
cv2_imshow(mask)


## Upscaling

In [ ]:
model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
url = "https://huggingface.co/spaces/jjourney1125/swin2sr/resolve/main/samples/butterfly.jpg"

image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
processor = Swin2SRImageProcessor()
pixel_values = processor(image, return_tensors="pt").pixel_values
with torch.no_grad():
  outputs = model(pixel_values)

In [ ]:
output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
output = np.moveaxis(output, source=0, destination=-1)
output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
Image.fromarray(output)